In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

    100% |████████████████████████████████| 993kB 20.9MB/s 
success!


In [0]:
from progressbar import ProgressBar, Percentage, Bar
from google.colab import files


In [0]:
import torch, pickle, os, sys, random, time
from torch import nn, optim


german = drive.CreateFile({'id': '17atuQ5p0LNY7nj4dcmlaFXD94dQV3NkB'})
german.GetContentFile('./german_no_pad_no_sorted.pickle') 
with open('./german_no_pad_no_sorted.pickle', 'rb') as f_in:
    german = pickle.load(f_in)

english = drive.CreateFile({'id': '1NEoxrv_6GeoM0fPrpw4RWiyO19FaZd20'})
english.GetContentFile('./english_no_pad_no_sorted.pickle') 
with open('./english_no_pad_no_sorted.pickle', 'rb') as f_in:
    english = pickle.load(f_in)

for i in range(len(german['train'])):
    german['train'][i] = torch.LongTensor(german['train'][i]).cuda()
    english['train'][i] = torch.LongTensor(english['train'][i]).cuda()
    
for i in range(len(german['dev'])):
    german['dev'][i] = torch.LongTensor(german['dev'][i]).cuda()
    english['dev'][i] = torch.LongTensor(english['dev'][i]).cuda()


In [0]:
def get_batches(german, english, b_sz):
    de_batches = [[]]
    
    # For every sentence in the dataset, add it to a batch, based on the batch size
    # if the sentence + current length is not greater than the batch size, 
    # then add it to the batch otherwise fill the current batch
    for sent in german:
        cur_len = 0
        for b in de_batches[-1]:
            cur_len += len(b)
    
        if (cur_len + len(sent)) <= b_sz: 
            de_batches[-1].append(sent)
        else:
            de_batches.append([])        
            de_batches[-1].append(sent)
    
    # For every batch within the entire set of batches, add padding to the sentences
    # that are less than the length of the longest sentence within the each batch.
    for b in de_batches:
        max_len = len(max(b, key=len))
        
        for sent in b:
            dif = max_len - len(sent)
            if dif > 0:
                pad_list = [0] * dif
                pad_list = torch.as_tensor(pad_list).to(device)
                sent.data = torch.cat((sent,pad_list), dim=0)

    # Within each batch, load the english sentence that corresponds to each
    # german sentence within that batch to a new english batch.
    en_batches = []
    k=0
    for i in range(len(de_batches)):
        tmp_batch = [0]*len(de_batches[i])
        for j in range(len(de_batches[i])):
            tmp_batch[j] = english[k]
            k+=1
        en_batches.append(tmp_batch)
        
    # Within each english batch, pad all the sentences to be the same length as the 
    # maximum sentence within that batch.
    for b in en_batches:
        max_len = len(max(b, key=len))
        
        for sent in b:
            dif = max_len - len(sent)
            if dif > 0:
                pad_list = [0] * dif
                pad_list = torch.as_tensor(pad_list).to(device)
                sent.data = torch.cat((sent,pad_list), dim=0)
        
    # Define the batches list, which contains an index for each batch. Within each index
    # of the batches list is a batch. Each batch is of length N (number of sentences), 
    # Each index (0..N) within the batch is a dictionary containing the source and target
    # sentence. Therefore, the batches list is a list of lists (essentially a 2D array), 
    # where each element in the list of lists/array is a dictionary containing the source
    # and target tensor.
    batches = []
    for i in range(len(de_batches)):
        dict_batch = []
        for j in range(len(de_batches[i])):
            tmp_dict = {"source": de_batches[i][j],
                       "target": en_batches[i][j]}
            dict_batch.append(tmp_dict)
        batches.append(dict_batch)
        
    return batches

In [0]:
def batchify(batch): # We need the batches to be of shape [sentence length, batch size]
    source_sent_len = (len(batch[0]['source']))
    target_sent_len = (len(batch[0]['target']))
    batch_size = (len(batch))
    
    source = torch.empty((source_sent_len, batch_size)).int().to(device)    
    target = torch.empty((target_sent_len, batch_size)).int().to(device)
    
    for i in range(len(batch)):
        source[:,i] = batch[i]['source']
        target[:,i] = batch[i]['target']
    
    return source, target


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Example:
#


In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        
        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden


class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
            
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]
        
        return torch.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim=2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim=1))
        
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0)


class Base(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.3):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).cuda()
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        output = trg[0,:]
        
        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


In [0]:
def create_model(english_data, german_data, device):    
    INPUT_DIM = len(german_data['idx2word'])
    OUTPUT_DIM = len(english_data['idx2word'])
    ENC_EMB_DIM = 256
    DEC_EMB_DIM = 256
    ENC_HID_DIM = 512
    DEC_HID_DIM = 512
    ENC_DROPOUT = 0.5
    DEC_DROPOUT = 0.5
    attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM,
                  DEC_HID_DIM, ENC_DROPOUT)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM,
                  DEC_HID_DIM, DEC_DROPOUT, attn)
    model = Base(enc, dec, device).to(device)
    return model

In [0]:
def write_to_drive(filename):
    from googleapiclient.discovery import build
    drive_service = build('drive', 'v3')

    from googleapiclient.http import MediaFileUpload

    file_metadata = {
      'name': filename
    }
    media = MediaFileUpload(filename, 
                            resumable=True)
    created = drive_service.files().create(body=file_metadata,
                                           media_body=media,
                                           fields='id').execute()
    print('File ID: {}'.format(created.get('id')))

In [0]:
def train(eng_train, eng_dev, de_train, de_dev, net, params):
  
    # padding is 0
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    
    #checkpoint = torch.load('checkpoint_3.pth')
    #net.load_state_dict(checkpoint['state_dict'])
    #optimizer.load_state_dict(checkpoint['optimizer'])
    tr_batches = get_batches(german['train'], english['train'], b_sz=params['batch_size'])
    print('batching done')
    
    
    
    for epoch in range(params['epochs']):
        random.shuffle(tr_batches)
        ep_loss = 0.
        start_time = time.time()
        pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=len(tr_batches)).start()
        
        
        # for each batch, calculate loss and optimize model parameters            
        i = 0
        for b in tr_batches: # TRAINING LOOP
            source, target = batchify(b)
            preds = net(source.long(), target.long())
            
            preds = preds[0:].view(-1, preds.shape[-1])
            eng_target = target[0:].view(-1)
            loss = criterion(preds, eng_target.long())
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss
            pbar.update(i+1)
            i += 1
        pbar.finish()
        
        checkpoint = {}
        checkpoint['state_dict'] = net.state_dict()
        checkpoint['optimizer'] = optimizer.state_dict()
        checkpoint['epoch'] = epoch
        file_name = 'checkpoint_{0}.pth'.format(epoch)        
        torch.save(checkpoint, file_name)
        write_to_drive(file_name)
        #files.download(file_name)
        
        print('epoch: {0}, loss: {1}, time: {2}'.format(epoch, ep_loss, time.time()-start_time))



In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = create_model(english, german, device)
params = {}
params['batch_size'] = 1000
params['epochs'] = 10
params['learning_rate'] = 0.001
train(english['train'], english['dev'],german['train'], german['dev'], model, params)

N/A%|                                                                         |

batching done


 98%|######################################################################## |

In [0]:
d = drive.CreateFile({'id': '1lhAkXEpQX9R2kCeduzpA2kP8ZV13gyyj'})
d.GetContentFile('./checkpoint.pth') 

In [0]:
def beam_search(probs, index, preds=[], end_index=3, beam_size=5):
    probs = torch.permute(1,0,2) 
    # [batch_size, seq_len, embeddings]
    soft = torch.softmax(dim=2)
    if len(preds) = 0:
        for i in range(beam_size):
            temp = torch.Tensor([2])
            preds.append(temp)
        
    
    

In [0]:

def evaluate(net):
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    eva_batches = get_batches(german['dev'], english['dev'], b_sz=1000)
    
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    
    checkpoint = torch.load('checkpoint.pth')
    net.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    ep_loss = 0
    i = 0
    all_preds = []
    for b in eva_batches: # TRAINING LOOP
        
        source, target = batchify(b)
        preds = net(source.long(), target.long(), 0)
        
        temp = preds[0:].view(-1, preds.shape[-1])
        eng_target = target[0:].view(-1)
        loss = criterion(temp, eng_target.long())
        preds = torch.softmax(preds, dim=2)
        preds = torch.argmax(preds, dim=2)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        ep_loss += loss
        all_preds.append(preds)
        i += 1
    print(i)
    return all_preds
   

In [0]:
a = evaluate(model)
eva_batches = get_batches(german['dev'], english['dev'], b_sz=1000)


171


172

In [0]:
eng_ex = []
for batch in a:
    x,y = batch.size()
    for i in range(y):
      sentence = []
      for idx in batch[:,i]:
        sentence.append(english['idx2word'][idx.item()])
        if idx.item() == 3:
           break
      eng_ex.append((' '.join(sentence[1:-1])))
      


In [0]:
t = a[batch_num]
x,y = t.size()
eng_ex = []
for i in range(y):
    sentence = []
    for idx in t[:,i]:
        sentence.append(english['idx2word'][idx.item()])
        if idx.item() == 3:
           break
    eng_ex.append((' '.join(sentence[1:-1])))

In [0]:
eng_t = []
for j in range(172):
  _, target = batchify(eva_batches[j])
  _, b = target.size()
  for i in range(b):
      sentence = []
      for idx in target[:,i]:
          sentence.append(english['idx2word'][idx.item()])
          if idx.item() == 3:
             break
      eng_t.append((' '.join(sentence[1:-1])))


In [0]:
# for i in range(len(eng_ex)):
#     print("Model: ", eng_ex[i], " Target: ", eng_t[i])
with open('real.out', 'w') as f:
    for line in eng_t:
        f.write(line + '\n')

In [0]:
!git clone https://github.com/moses-smt/mosesdecoder.git
!pip install sacrebleu

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 147295, done.
remote: Total 147295 (delta 0), reused 0 (delta 0), pack-reused 147295
Receiving objects: 100% (147295/147295), 129.69 MiB | 12.70 MiB/s, done.
Resolving deltas: 100% (113833/113833), done.
  Stored in directory: /root/.cache/pip/wheels/56/c0/fb/1c7f9b3a71f64cdf86291cc645596f71746807bf2f72b3c1dd
Successfully built sacrebleu


In [0]:
%%shell
#!/bin/bash

# This is a reference to the gold translations from the dev set
REFERENCE_FILE="real.out"

# XXX: Change the following line to point to your model's output!
TRANSLATED_FILE="model.out"

# The model output is expected to be in a tokenized form. Note, that if you
# tokenized your inputs to the model, then simply joined each output token with
# whitespace you should get tokenized outputs from your model.
# i.e. each output token is separate by whitespace
# e.g. "My model 's output is interesting ."
perl "mosesdecoder/scripts/tokenizer/detokenizer.perl" -l en < "$TRANSLATED_FILE" > "$TRANSLATED_FILE.detok"

PARAMS=("-tok" "intl" "-l" "de-en" "$REFERENCE_FILE")
sacrebleu "${PARAMS[@]}" < "$TRANSLATED_FILE.detok"

Detokenizer Version $Revision: 4134 $
Language: en
BLEU+case.mixed+lang.de-en+numrefs.1+smooth.exp+tok.intl+version.1.3.1 = 9.5 42.4/15.0/6.1/2.6 (BP = 0.954 ratio = 0.955 hyp_len = 155769 ref_len = 163089)


Thu Apr 25 13:52:53 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  